In [186]:
from splunk_connect import SplunkConnection
from splunk_search import SplunkSearch

In [187]:
import splunklib.client as splunkclient

class SplunkConnection:
    DEBUG = False

    def __init__(self):
        'create a Splunk service instance and log in'
        self.service = ""
        self.kwargs = self.get_credentials_from_rc()
        if self.DEBUG:
            print (self.kwargs)
        # connect requires keywords not positional args
        self.service = splunkclient.connect(**self.kwargs)
        if self.DEBUG:
            print ('service: ' , self.service)

    def get_credentials_from_rc(self):
        'read connection parameters from an rc file'
        '''to use: create a splunkrc file that has username, host, etc:
           username = somename
           host = myhost
           password = somepass
           ...
        '''
        kwargs = {}
        with open("c:\\projects\\python\splunk_py\\.splunkrc") as myfile:
            for line in myfile:
                name, var = line.partition("=")[::2]
                # by convention python comm likes to use kwargs as the
                # name of key word sweeper dictionary
                kwargs[name] = var.rstrip()
        return kwargs

    def get_app_names(self):
        'print installed apps to the console to verify login'
        app_list = []
        for app in self.service.apps:
            app_list.append(app.name)
        return app_list


In [4]:
import splunklib.results as results

class SplunkSearch:
    DEBUG = True

    def __init__(self,service):
        'create a Splunk service instance and log in'
        self.job = ''
        self.jobs = service.jobs

    def set_blocking_search(self,query, num_results):
        'place a blocking search on jobs collection'
        # num results can be 0 for no limit, or limited by num_results

        if num_results == 0:
            search = query
        else:
            search = query + ' | head ' + str(num_results)

        if self.DEBUG:
            print ('Query: ', search)

        kwargs = {"exec_mode": "blocking"}
        self.job = self.jobs.create(search, **kwargs)

    def set_search(self,num_results):
        pass


    def get_search_results(self):
        rr = results.ResultsReader(self.job.results())
        return rr




In [188]:
import pandas as pd
def driver():
    # create a connection to Splunk
    sc = SplunkConnection()
    # run a search
    ss = SplunkSearch(sc.service)
    # define the search in SPL
    query = 'search index=sysmon source="WinEventLog:Microsoft-Windows-Sysmon/Operational" | fields Hashes | fields - _*'
    # limit to 10 results (just for demo speed) and set the search
    ss.set_blocking_search(query,10)
    # set to a dataframe
    d = []
    for result in ss.get_search_results():
        d.append(result)
    df = pd.DataFrame(d)
    return df
  

In [189]:
df = driver()
df['SHA256']= df.Hashes.str.split(',').str[1].str.split('=').str[1]
df.SHA256

Query:  search index=sysmon source="WinEventLog:Microsoft-Windows-Sysmon/Operational" | fields Hashes | fields - _* | head 10


0    B70E8B7DDC7DC9AD850547776DCEC48E829959C10449FD...
1    670E3064127C622A748220249C427404B96BF29A930320...
2    34E5393E7ED503F97AC8E0E327E79590F46AA87BFF4FB1...
3    29F00C4D8D36CC701B8D903C2341119A9BC2D614BDD8A6...
4    994811D828541A10D8B0265D325CE111959F3D9D234F90...
5    29F00C4D8D36CC701B8D903C2341119A9BC2D614BDD8A6...
6    B282C60A794135C34E8CCA5BE765035BE365C331B91A18...
7    B70E8B7DDC7DC9AD850547776DCEC48E829959C10449FD...
8    34E5393E7ED503F97AC8E0E327E79590F46AA87BFF4FB1...
9    B282C60A794135C34E8CCA5BE765035BE365C331B91A18...
Name: SHA256, dtype: object

In [190]:
df['MD5']= df.Hashes.str.split(',').str[0].str.split('=').str[1]
df.MD5

0    465E59D0BE10B7FD0582C69D9F8F755F
1    201EF3ED68CBFFE77D099EC45B8B6ECA
2    AC9AF23748D45A2AB30D4E3F176F9EC3
3    726130E34D51AC726352EA31DD26D8EC
4    E47C50BADC1A0BDCF711A618F605146B
5    726130E34D51AC726352EA31DD26D8EC
6    164DBC480641B21FA928A4022FAD5FD8
7    465E59D0BE10B7FD0582C69D9F8F755F
8    AC9AF23748D45A2AB30D4E3F176F9EC3
9    164DBC480641B21FA928A4022FAD5FD8
Name: MD5, dtype: object

In [191]:
import requests
import pandas as pd

class HashChecker():
    '''
    The HashChecker object is used  to process, arrange and store VirusTotal API calls
    against a file hash - the results are neatly arranged in a Pandas DataFrame
    
    
    hash : str
    This argument is used to pass the single hash in question 
    
    Todo:
    create the ability to pass multiple hashes
    '''

    
    def __init__(self, hash):
    
        self.hash = hash
        self.myparams= {}
        
        # HTTP Request results
        self.request_results = []
        
        # VirusTotal settings
        self.filename = 'c:\\projects\\python\splunk_py\\.virustotalrc'
        self.url = 'https://www.virustotal.com/vtapi/v2/file/report'
        
        # process the hash
        self.get_params()
        # Get the results of hash check from Request call
        self.request_results = self.get_results()
        
        # use a dataframe for flexibility and scalability
        self.df = pd.DataFrame()
        self.set_results_dataframe2()
        
    def get_results(self):
        '''
        Method to call the REST API and check a single hash
        '''
        data = requests.get(self.url, params=self.myparams)
        return data

    def get_params(self):
        '''
        Method to get your hidden key from an ini file
        read the values from a file with format
        apikey=<yourapikey>
        '''
        with open(self.filename) as myfile:
            for line in myfile:
                key, value = line.partition("=")[::2]
                self.myparams[key.strip()] = value.strip()
                
    def set_results_dataframe(self):
        '''
        Method to create a dataframe with results using format
                    columns
            rows
            service1 detected   result  update  version
            service2 detected   result  update  version
        '''
        # use a dataframe that gets the normalized json from Request result
        # notice we only want the 'scans' level of the json
        self.df = json_normalize(self.request_results.json().get('scans'))
        
        # to aid in re-arranging more logically, tranpose rows and columns
        self.df = self.df.transpose()
    
        # create a holding dataframe in order to re-arrange the 
        # columns more logically
        ndf = pd.DataFrame(columns=['detected', 'result', 'update', 'version', ])    
        for i in range(0,len(self.df),4):
            ndf=ndf.append({ 'detected' : self.df.index[i].split('.')[0],
                             'result': self.df.iloc[i+1,0], 'update': self.df.iloc[i+2,0],
                             'version' : self.df.iloc[i+3,0]}, ignore_index=True)
        
        # reset the object dataframe to this new arrangment
        self.df = ndf
        
    def set_results_dataframe2(self):
        self.df = pd.DataFrame.from_dict(hash_results.get_results().json()['scans'], orient='index').reset_index()
        self.df.rename(columns={'index':'service'},inplace=True)

        

In [195]:

# hash of mimikatz
resource = 'BCD703932C9BEDE27659407E458AE103D0B4CC88'
hash_results = HashChecker(resource)
print(hash_results.request_results)
print(hash_results.df.head(6))




<Response [200]>
    service      version detected    update                      result
0     ALYac      1.1.1.5    False  20190831                        None
1      APEX         5.57     True  20190830                   Malicious
2       AVG  18.4.3895.0     True  20190831        FileRepMalware [PUP]
3   Acronis     1.0.1.51    False  20190829                        None
4  Ad-Aware    3.0.5.370     True  20190901  Gen:Application.Mimikatz.2
5  AegisLab          4.2     True  20190831   Trojan.Win64.Mimikatz.i!c


In [145]:
print(hash_results.request_results.status_code)

200


In [137]:
import json
jdata = json.dumps(hash_results.get_results().json(), indent = 4)
print (jdata)

{
    "scans": {
        "Zoner": {
            "version": "1.0.0.1",
            "detected": false,
            "update": "20190825",
            "result": null
        },
        "CAT-QuickHeal": {
            "version": "14.00",
            "detected": true,
            "update": "20190825",
            "result": "Trojanpws.Win64"
        },
        "Symantec": {
            "version": "1.10.0.0",
            "detected": true,
            "update": "20190824",
            "result": "Hacktool.Mimikatz"
        },
        "VBA32": {
            "version": "4.0.0",
            "detected": true,
            "update": "20190823",
            "result": "TrojanPSW.Win64.Mimikatz"
        },
        "MicroWorld-eScan": {
            "version": "14.0.297.0",
            "detected": true,
            "update": "20190826",
            "result": "Gen:Application.Mimikatz.2"
        },
        "Yandex": {
            "version": "5.5.2.24",
            "detected": true,
            "update": "201

In [193]:
print(hash_results.get_results().json()['scans']['AVG']['detected'])

True


In [194]:
print(type(jdata))
print(type(hash_results.get_results().json()))

<class 'str'>
<class 'dict'>
